## Financial Fraud Classifier
#### Sam Berkson and Ben Puryear
#### CPSC 322 Final Project

In [2]:
from mysklearn import myclassifiers, myevaluation, mypytable, myutils
import importlib

### Introduction)  

* Why are we doing this in the first place?
    * The potential impact of our results are pretty intuitive.  Automated fraud detection systems have massive applications for both businesses as well as financial institutions.  A system like this could be used to stop fradulent activies, and help identify the perpetrators of financial crimes.  A system like ours would likely affect the following stakeholders:
    * Consumers
        * Consumer's financial assets gain increased security, and helps to increase trust in modern financial institutions which can help stimulate economic growth.
    * Businesses
        * Businesses can save needless expense by ensuring that they are not the victims of fradulent transactions out of company accounts, or the victims of fradulent orders.
    * Financial Institutions
        * Banks can automate the task of consumer protection, allowing them greater resources to pursue recovery of funds through all available avenues.
    * Regulatory Agencies
        * Regulatory agencies can work with financial institutions to share identifying information regarding fradulent transactions, and use this information to pursue recovery of funds through a variety of methods.  One cool method is through appoint a receiver to recover and equitably dispense recovered funds among secured and unsecured creditors, which Sam just so happens to do for work over the summer!
    * Financial Criminals
        * This gang isnt so lucky.  Criminals only gain determent from committing felonies and robbing consumers and instituions of their assets.

    * Personal interest:
        * Sam works with financial fraud on a daily basis for work

![image](media/saftey.jpeg)

* Dataset
    * Our dataset originates from Kaggle, coming packaged as a csv file.  This csv file contains just over a million instances of different types of financial transactions from different accounts, and marks whether or not the transaction was flagged as fradulent or not, and whether or not the transaction was actually fradulent.  Our dataset contains 11 attributes:
 
    * step
        * Step maps to a number of hours, where 1 step is 1 hour
     * type
        * This identifies the type of transaction.  It can be: CASH-IN; CASH-OUT; DEBIT; PAYMENT; and TRANSFER.
        * This could be a pretty useful attribute to use in classification.
    * amount
        * This is the amount of money transferred (in the local currency).
        * This can also be a pretty useful attribute to use in our classification.
    * nameOrig
        * This identifies the customer who initiated the transaction.
    * oldBalanceOrg
        * This is the initial balance before the transaction.
    * newBalanceOrig
        * This is the new balance after the transaction.
    * nameDest\n",
        * This identifies the recipient of the transfer.
    * oldBalanceDest
        * This is the initial balance recipient before the transaction
    * newBalanceDest
        * This is the new balance recipient after the transaction.
        * This can be a useful tool for classification when used in comparison to oldBalanceDest for any given instance.
    * isFraud
        * This is the transactions made by the fraudulent agents inside the simulation. In this specific dataset the fraudulent behavior of the agents aims to profit by taking control or customers accounts and try to empty the funds by transferring to another account and then cashing out of the system.
    * isFlaggedFraud
        * The business model aims to control massive transfers from one account to another and flags illegal attempts. An illegal attempt in this dataset is an attempt to transfer more than 200.000 in a single transaction.
    
Our classification goal is to correctly predict whether any given transaction is fradulent or not.

### EDA)

### Classification)

### Conclusion)